In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 3

## 로드

In [5]:
import pandas as pd
import numpy as np

Olist = pd.read_csv('/content/drive/MyDrive/Adidas&E-commerce DA/List of Orders.csv')
Detail = pd.read_csv('/content/drive/MyDrive/Adidas&E-commerce DA/Order Details.csv')

영수증을 사용하는 PoS 데이터

Olist 는 장바구니? 구매? 라고 보면 되고 (Order ID 단위 하나가 장바구니 한번 결제할 때 마다 의 개념)

Detail 은 영수증이라고 생각하면 될 듯

==

주문번호/날짜/고객명/배송지정보

In [6]:
Olist.head()

,Order ID,Order Date,CustomerName,State,City
0,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad
1,B-25602,01-04-2018,Pearl,Maharashtra,Pune
2,B-25603,03-04-2018,Jahan,Madhya Pradesh,Bhopal
3,B-25604,03-04-2018,Divsha,Rajasthan,Jaipur
4,B-25605,05-04-2018,Kasheen,West Bengal,Kolkata


주문번호/판매금액/이익/수량/카테고리/서브카테고리

In [7]:
Detail.head()

,Order ID,Amount,Profit,Quantity,Category,Sub-Category
0,B-25601,1275.0,-1148.0,7,Furniture,Bookcases
1,B-25601,66.0,-12.0,5,Clothing,Stole
2,B-25601,8.0,-2.0,3,Clothing,Hankerchief
3,B-25601,80.0,-56.0,4,Electronics,Electronic Games
4,B-25602,168.0,-111.0,2,Electronics,Phones


Olist는 560개인데 500개 값만 있음

Olist는 전부 object

날짜는 datetime으로


In [8]:
Olist.info(), Detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Order ID      500 non-null    object
 1   Order Date    500 non-null    object
 2   CustomerName  500 non-null    object
 3   State         500 non-null    object
 4   City          500 non-null    object
dtypes: object(5)
memory usage: 22.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order ID      1500 non-null   object 
 1   Amount        1500 non-null   float64
 2   Profit        1500 non-null   float64
 3   Quantity      1500 non-null   int64  
 4   Category      1500 non-null   object 
 5   Sub-Category  1500 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 70.4+ KB


(None, None)

In [9]:
print(Detail[Detail['Profit'] < 0].shape)
print(Detail[Detail['Profit'] == 0].shape)
print(Detail[Detail['Profit'] > 0].shape)

(503, 6)
(47, 6)
(950, 6)


Profit이 마이너스인 것들은 환불로 보여짐. or 그 외

목적에 따라

1. 환불이 중요하지 않고 고객이 구매했다는 사실이 중요한 경우 : 마이너스(-) 값 drop시켜버리고 +만 살려서 분석하는 경우도 있음

2. 매출액이나 이익 중요한 경우 : 100원 구매했을 때 -100 해서 0으로 만들어서 분석함

## 전처리

모든 칼럼이 Na 면 그냥 지워버리면 편함.

하지만 지그재그 처럼 .. =>   1 NA 3 4, NA 2 5 NA   //  1 2 3 4 , NA 1 NA NA  결측치가 이렇게 있으면 두 경우 전부 NA를 4개로 인식하기 때문에 직접 확인해 볼 필요가 있음

실제로 사용할 때는 이걸 함수를 하나 만들어서 column별로 루프를 돌아가면서 작업을 하는 경우가 있음

In [10]:
Olist[Olist['Order ID'].isna()].isnull() #.sum()

,Order ID,Order Date,CustomerName,State,City
500,True,True,True,True,True
501,True,True,True,True,True
502,True,True,True,True,True
503,True,True,True,True,True
504,True,True,True,True,True
505,True,True,True,True,True
506,True,True,True,True,True
507,True,True,True,True,True
508,True,True,True,True,True
509,True,True,True,True,True


In [11]:
Olist = Olist[Olist['Order ID'].isna() == False]
Olist.shape

(500, 5)

In [12]:
data = Olist.merge(Detail, on='Order ID')
data.shape

(1500, 10)

merge를 했을 때 확인해야되는 것
1. 데이터의 누락. merge하면서 누락된 것이 있는가
2. 혹시 중복이 생기지 않았는가

In [13]:
print(len(Olist['Order ID'].unique()))
print(len(data['Order ID'].unique()))
print(len(Detail['Order ID'].unique()))

500
500
500


In [14]:
# 결측치 확인, True가 하나라도 있으면 False가 안뜸
data.isnull().values.any()

False

In [15]:
data.head()

,Order ID,Order Date,CustomerName,State,City,Amount,Profit,Quantity,Category,Sub-Category
0,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,1275.0,-1148.0,7,Furniture,Bookcases
1,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,66.0,-12.0,5,Clothing,Stole
2,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,8.0,-2.0,3,Clothing,Hankerchief
3,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,80.0,-56.0,4,Electronics,Electronic Games
4,B-25602,01-04-2018,Pearl,Maharashtra,Pune,168.0,-111.0,2,Electronics,Phones


In [16]:
# 날짜 변환
data['Order Date'] = pd.to_datetime(data['Order Date'], format='%d-%m-%Y')

In [17]:
data['Order Date'].min(), data['Order Date'].max()

(Timestamp('2018-04-01 00:00:00'), Timestamp('2019-03-31 00:00:00'))

In [18]:
data['year'] = data['Order Date'].dt.year
data['month'] = data['Order Date'].dt.month
data['yearmonth'] = data['Order Date'].astype('str').str.slice(0,7)

In [19]:
data.head(1)

,Order ID,Order Date,CustomerName,State,City,Amount,Profit,Quantity,Category,Sub-Category,year,month,yearmonth
0,B-25601,2018-04-01,Bharat,Gujarat,Ahmedabad,1275.0,-1148.0,7,Furniture,Bookcases,2018,4,2018-04


## EDA (plotly 이용)

https://plotly.com/python/

plotly 는 Interative함
1. 마우스 갖다대면 값이 뜸
2. 그래프 축소/확대
3. 기능 자체가 matplotlib 보다 좋음

In [20]:
import plotly.express as px

### 판매 분석

In [21]:
data.head()

,Order ID,Order Date,CustomerName,State,City,Amount,Profit,Quantity,Category,Sub-Category,year,month,yearmonth
0,B-25601,2018-04-01,Bharat,Gujarat,Ahmedabad,1275.0,-1148.0,7,Furniture,Bookcases,2018,4,2018-04
1,B-25601,2018-04-01,Bharat,Gujarat,Ahmedabad,66.0,-12.0,5,Clothing,Stole,2018,4,2018-04
2,B-25601,2018-04-01,Bharat,Gujarat,Ahmedabad,8.0,-2.0,3,Clothing,Hankerchief,2018,4,2018-04
3,B-25601,2018-04-01,Bharat,Gujarat,Ahmedabad,80.0,-56.0,4,Electronics,Electronic Games,2018,4,2018-04
4,B-25602,2018-04-01,Pearl,Maharashtra,Pune,168.0,-111.0,2,Electronics,Phones,2018,4,2018-04


In [22]:
df = data.groupby('yearmonth').agg({'Quantity' : 'sum'}).reset_index()
df

,yearmonth,Quantity
0,2018-04,389
1,2018-05,423
2,2018-06,369
3,2018-07,240
4,2018-08,446
5,2018-09,331
6,2018-10,419
7,2018-11,578
8,2018-12,412
9,2019-01,745


In [23]:
fig1 = px.line(df, x='yearmonth', y='Quantity', title='Sales Quantity by month')
fig1.show()

In [24]:
df2 = data.groupby('yearmonth').agg({'Amount' : 'sum'}).reset_index()
fig2 = px.line(df2, x='yearmonth', y='Amount', title='Sales Amount by month')
fig2.show()

판매금액과 수량 상관관계 높을 것 같음

시계열적으로 봤을 때 증가하는 트렌드가 있다.

In [25]:
df3 = data.pivot_table(index='Category', values='Quantity', aggfunc='sum').reset_index()
df3

,Category,Quantity
0,Clothing,3516
1,Electronics,1154
2,Furniture,945


In [26]:
fig3 = px.bar(df3, x='Category', y='Quantity')
fig3.show()

In [27]:
df4 = data.pivot_table(index=['yearmonth', 'Category'], values='Quantity', aggfunc='sum').reset_index()
df4.head()

,yearmonth,Category,Quantity
0,2018-04,Clothing,251
1,2018-04,Electronics,75
2,2018-04,Furniture,63
3,2018-05,Clothing,233
4,2018-05,Electronics,116


In [28]:
fig4 = px.bar(df4, x='yearmonth', y='Quantity', color='Category')
fig4.show()

### 지역별 주력 판매상품 분석 - (heat map)

In [29]:
df5 = data.pivot_table(index=['State', 'Sub-Category'], values=['Quantity', 'Amount', 'Profit'], aggfunc='sum').reset_index()
df5.head()

,State,Sub-Category,Amount,Profit,Quantity
0,Andhra Pradesh,Accessories,231.0,22.0,2
1,Andhra Pradesh,Bookcases,4581.0,-1642.0,25
2,Andhra Pradesh,Electronic Games,188.0,-193.0,2
3,Andhra Pradesh,Furnishings,57.0,7.0,3
4,Andhra Pradesh,Hankerchief,710.0,80.0,32


In [30]:
fig5 = px.density_heatmap(df5, x='State', y='Sub-Category', z='Quantity', title='Quantity heatmap')
fig6 = px.density_heatmap(df5, x='State', y='Sub-Category', z='Amount', title='Amount heatmap')
fig5.show(), fig6.show()

(None, None)

# 4

In [31]:
!pip install streamlit
!npm install localtunnel
!pip install "ipywidgets>=7, <8"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.776s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit 

### 코드 함수 모듈화

In [32]:
def load_data():

  Olist = pd.read_csv('/content/drive/MyDrive/Adidas&E-commerce DA/List of Orders.csv')
  Detail = pd.read_csv('/content/drive/MyDrive/Adidas&E-commerce DA/Order Details.csv')
  data = Olist.merge(Detail, on='Order ID')

  return data

def preproc(): # 전처리
  data['Order Date'] = pd.to_datetime(data['Order Date'], format='%d-%m-%Y')
  data['year'] = data['Order Date'].dt.year
  data['month'] = data['Order Date'].dt.month
  data['yearmonth'] = data['Order Date'].astype('str').str.slice(0,7)

  return data

In [33]:
def line_chart(data, x, y, title):
  df = data.groupby(x).agg({y : 'sum'}).reset_index()
  fig = px.line(df, x=x, y=y, title=title)
  fig.show()

  return fig

In [34]:
def bar_chart(data, x, y, color=None):
  if color is not None:
    index = [x, color]
  else:
    index = x

  df = data.pivot_table(index=index, values=y, aggfunc='sum').reset_index()
  fig = px.bar(df, x=x, y=y, color=color)
  fig.show()

  return fig

In [35]:
def heatmap(data, z, title):
  df = data.pivot_table(index=['State', 'Sub-Category'], values=['Quantity', 'Amount', 'Profit'], aggfunc='sum').reset_index()
  fig = px.density_heatmap(df, x='State', y='Sub-Category', z=z, title=title)
  fig.show()

  return fig

### streamlit 이용 대시보드 구현

%%writefile app.py : 셀에 있는 내용이 들어간 app.py 파일을 만들어라 (%% : 주피터노트북 커맨드)

@st.cache_data : 데코레이터
- load_data 하면 csv파일을 어플리케이션 앱으로 불러와서 메모리에 로드 -> 전처리 -> 시각화 -> 웹
- linechart, barchart 버튼 2개가 있다 가정하면 line 그렸다가 bar 그리면 로드, 전처리 과정을 또 해야됨(시간소요ㅠ)
- 결과 데이터를 데코레이터가 메모리 캐시 영역에 가지고 있게 함. (서비스의 영역이기 때문에 속도가 중요함)



In [43]:
%%writefile app.py


import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np

# Data Load
@st.cache_data
def load_data():

  Olist = pd.read_csv('/content/drive/MyDrive/Adidas&E-commerce DA/List of Orders.csv')
  Detail = pd.read_csv('/content/drive/MyDrive/Adidas&E-commerce DA/Order Details.csv')
  data = Olist.merge(Detail, on='Order ID')

  return data

# Preprocessing
def preproc():
  data['Order Date'] = pd.to_datetime(data['Order Date'], format='%d-%m-%Y')
  data['year'] = data['Order Date'].dt.year
  data['month'] = data['Order Date'].dt.month
  data['yearmonth'] = data['Order Date'].astype('str').str.slice(0,7)

  return data

# =============================================

# plotly graph
def line_chart(data, x, y, title):
  df = data.groupby(x).agg({y : 'sum'}).reset_index()
  fig = px.line(df, x=x, y=y, title=title)
  fig.show()

  return df, fig

def bar_chart(data, x, y, color=None):
  if color is not None:
    index = [x, color]
  else:
    index = x

  df = data.pivot_table(index=index, values=y, aggfunc='sum').reset_index()
  fig = px.bar(df, x=x, y=y, color=color)
  fig.show()

  return fig

def heatmap(data, z, title):
  df = data.pivot_table(index=['State', 'Sub-Category'], values=['Quantity', 'Amount', 'Profit'], aggfunc='sum').reset_index()
  fig = px.density_heatmap(df, x='State', y='Sub-Category', z=z, title=title)
  fig.show()

  return fig

if __name__ == "__main__":

  st.title('E-commerce Data Analysis')

  # Data Load
  data = load_data()
  # Data Preprocessing
  data = preproc()

st.subheader('월별 판매량 분석')
with st.form('form', clear_on_submit = True): # form : 영역을 나누는 부분
  col1, col2 = st.columns(2)
  submitted1 = col1.form_submit_button('판매량 그래프')
  submitted2 = col2.form_submit_button('매출액 그래프')

  # When clicked
  if submitted1:
    df1, fig1 = line_chart(data, 'yearmonth', 'Quantity', 'Sales Quantity by month') # <---------------------------------------- error
    st.dataframe(df1.T)
    st.plotly_chart(fig1, theme="streamlit", use_container_width=True)
  elif submitted2:
    df2, fig2 = line_chart(data, 'yearmonth', 'Amount', 'Sales Quantity by month') # <---------------------------------------- error
    st.dataframe(df2.T)
    st.plotly_chart(fig2, theme="streamlit", use_container_width=True)

st.subheader('품목별 판매량')
col1, col2 = st.columns(2)
with col1:
  col1.subheader('카테고리별 판매량')
  fig3 = bar_chart(data, 'Category', 'Quantity')
  st.plotly_chart(fig3, theme='streamlit', use_container_width=True)
with col2:
  col2.subheader('월별/카테고리별 누적 차트')
  fig4 = bar_chart(data, 'yearmonth', 'Quantity', 'Category')
  st.plotly_chart(fig4, theme='streamlit', use_container_width=True)

st.subheader('지역별 주력 판매상품')
tab1, tab2 = st.tabs(['Quantity heat map', 'Amount heat map'])
with tab1:
  fig5 = heatmap(data, 'Quantity', 'Quantity heat map')
  st.plotly_chart(fig5, theme='streamlit', use_container_width=True)
with tab2:
  fig6 = heatmap(data, 'Amount', 'Amount heat map')
  st.plotly_chart(fig6, theme='streamlit', use_container_width=True)

Overwriting app.py


In [ ]:
!streamlit run /content/app.py & npx lt --port 8501